In [1]:
import xarray as xr
import numpy as np
import numpy as np

In [2]:
zarr_dataset = xr.open_zarr('modulus/examples/weather/unified_recipe/unified_recipe_datasets/arco_era5.zarr')

In [18]:
zarr_dataset['temperature']['time'].values.tolist()

[1514764800000000000,
 1514786400000000000,
 1514808000000000000,
 1514829600000000000,
 1514851200000000000,
 1514872800000000000,
 1514894400000000000,
 1514916000000000000,
 1514937600000000000,
 1514959200000000000,
 1514980800000000000,
 1515002400000000000,
 1515024000000000000,
 1515045600000000000,
 1515067200000000000,
 1515088800000000000,
 1515110400000000000,
 1515132000000000000,
 1515153600000000000,
 1515175200000000000,
 1515196800000000000,
 1515218400000000000,
 1515240000000000000,
 1515261600000000000,
 1515283200000000000,
 1515304800000000000,
 1515326400000000000,
 1515348000000000000,
 1515369600000000000]

In [20]:
lon_grid, lat_grid = np.meshgrid(zarr_dataset.longitude, zarr_dataset.latitude)

# Convert grids to radians
lon_rad_grid = np.deg2rad(lon_grid)
lat_rad_grid = np.deg2rad(lat_grid)

# Calculate spherical coordinates
cos_lat_grid = np.cos(lat_rad_grid)
sin_lon_grid = np.sin(lon_rad_grid)
cos_lon_grid = np.cos(lon_rad_grid)

# Expand grids to include the time dimension
time_len = len(zarr_dataset.time)
cos_lat_grid_time = np.tile(cos_lat_grid, (time_len, 1, 1))
sin_lon_grid_time = np.tile(sin_lon_grid, (time_len, 1, 1))
cos_lon_grid_time = np.tile(cos_lon_grid, (time_len, 1, 1))
lon_grid_time = np.tile(lon_grid, (time_len, 1, 1))
lat_grid_time = np.tile(lat_grid, (time_len, 1, 1))

# Add the grids as data variables
zarr_dataset = zarr_dataset.assign(
    {
        "cos_latitude": (("time", "latitude", "longitude"), cos_lat_grid_time),
        "cos_longitude": (("time", "latitude", "longitude"), cos_lon_grid_time),
        "sin_longitude": (("time", "latitude", "longitude"), sin_lon_grid_time),        
    }
)


In [5]:
forcing_variables=('toa_incident_solar_radiation', 'year_progress_sin', 'year_progress_cos', 'day_progress_sin', 'day_progress_cos')
DAY_PROGRESS = "day_progress"
YEAR_PROGRESS = "year_progress"
_DERIVED_VARS = {
    DAY_PROGRESS,
    f"{DAY_PROGRESS}_sin",
    f"{DAY_PROGRESS}_cos",
    YEAR_PROGRESS,
    f"{YEAR_PROGRESS}_sin",
    f"{YEAR_PROGRESS}_cos",
}

In [6]:
# zarr_dataset['batch'] = 1
# zarr_dataset = zarr_dataset.set_coords('batch')
zarr_dataset = zarr_dataset.expand_dims({'batch':1})
datetime = zarr_dataset.time.values

datetime_2d = np.array([datetime])
zarr_dataset = zarr_dataset.assign_coords({'datetime': (('batch', 'time'), datetime_2d)})

In [7]:
import zarr
import data_utils, solar_radiation

data_utils.add_derived_vars(zarr_dataset)
# data_utils.add_tisr_var(zarr_dataset)

In [8]:
zarr_dataset = zarr_dataset.drop_vars(["datetime", 'year_progress', 'day_progress'])

In [13]:
zarr_dataset = zarr_dataset.squeeze('batch', drop=True)

In [24]:
zarr_dataset.to_zarr('modulus/examples/weather/unified_recipe/unified_recipe_datasets/arco_era5.zarr')

In [25]:
zarr_dataset

<xarray.Dataset> Size: 11GB
Dimensions:                       (time: 29, latitude: 721, longitude: 1440,
                                   level: 13)
Coordinates:
  * latitude                      (latitude) float32 3kB 90.0 89.75 ... -90.0
  * level                         (level) int64 104B 50 100 150 ... 850 925 1000
  * longitude                     (longitude) float32 6kB 0.0 0.25 ... 359.8
  * time                          (time) datetime64[ns] 232B 2018-01-01 ... 2...
Data variables: (12/21)
    10m_u_component_of_wind       (time, latitude, longitude) float32 120MB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    10m_v_component_of_wind       (time, latitude, longitude) float32 120MB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    2m_temperature                (time, latitude, longitude) float32 120MB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    cos_latitude                  (time, latitude, longitude) float32 120MB -...
    cos_longitude                 (time, latitude, longitude) float32 120MB 1...
    day_progress_cos              (time, longitude) float32 167kB dask.array<chunksize=(29, 1440), meta=np.ndarray>
    ...                            ...
    u_component_of_wind           (time, level, latitude, longitude) float32 2GB dask.array<chunksize=(1, 1, 721, 1440), meta=np.ndarray>
    v_component_of_wind           (time, level, latitude, longitude) float32 2GB dask.array<chunksize=(1, 1, 721, 1440), meta=np.ndarray>
    vertical_velocity             (time, level, latitude, longitude) float32 2GB dask.array<chunksize=(1, 1, 721, 1440), meta=np.ndarray>
    year_progress_cos             (time) float32 116B dask.array<chunksize=(29,), meta=np.ndarray>
    year_progress_sin             (time) float32 116B dask.array<chunksize=(29,), meta=np.ndarray>
    sin_longitude                 (time, latitude, longitude) float32 120MB 0...
Attributes:
    last_updated:           2025-03-04 03:40:28.857682+00:00
    valid_time_start:       1940-01-01
    valid_time_stop:        2024-11-30
    valid_time_stop_era5t:  2025-02-26

In [13]:
curated_train = xr.open_zarr('modulus/examples/weather/unified_recipe/unified_recipe_datasets/graphcast_fixed_train.zarr')

In [14]:
curated_train

<xarray.Dataset> Size: 3GB
Dimensions:                (time: 5, constants_channel: 2, latitude: 721,
                            longitude: 1440, forcings_channel: 5,
                            inputs_channel: 11, level: 13,
                            node_features_channel: 4)
Coordinates:
  * constants_channel      (constants_channel) <U23 184B 'land_sea_mask' 'geo...
  * forcings_channel       (forcings_channel) <U28 560B 'toa_incident_solar_r...
  * inputs_channel         (inputs_channel) <U23 1kB '2m_temperature' ... 'sp...
  * latitude               (latitude) float32 3kB 90.0 89.75 ... -89.75 -90.0
  * level                  (level) int64 104B 50 100 150 200 ... 850 925 1000
  * longitude              (longitude) float32 6kB 0.0 0.25 0.5 ... 359.5 359.8
  * node_features_channel  (node_features_channel) <U19 304B 'total_precipita...
  * time                   (time) datetime64[ns] 40B 2018-01-01 ... 2018-01-02
Data variables:
    constants              (time, constants_channel, latitude, longitude) float32 42MB dask.array<chunksize=(1, 2, 721, 1440), meta=np.ndarray>
    forcings               (time, forcings_channel, latitude, longitude) float32 104MB dask.array<chunksize=(1, 5, 721, 1440), meta=np.ndarray>
    inputs                 (time, inputs_channel, latitude, longitude, level) float32 3GB dask.array<chunksize=(1, 11, 721, 1440, 1), meta=np.ndarray>
    node_features          (time, node_features_channel, latitude, longitude) float32 83MB dask.array<chunksize=(1, 4, 721, 1440), meta=np.ndarray>
Attributes:
    tp_channel_index:  6

In [6]:
import pandas as pd
import xarray as xr
import numpy as np

def check_for_nulls(dataset: xr.Dataset):
    """Checks an xarray Dataset for null values (NaNs and NaTs).

    Args:
        dataset: The xarray Dataset to check.

    Returns:
        A dictionary indicating where nulls were found.
    """

    results = {}

    # 1. Check Data Variables
    results["data_variables"] = {}
    for var_name in dataset.data_vars:
        null_count = dataset[var_name].isnull().sum()  # Get the Dask array
        null_count = null_count.compute()  # Compute the result
        #null_count = null_count.item() # We don't need .item anymore
        results["data_variables"][var_name] = null_count > 0
        if null_count > 0:
            results["data_variables"][f"{var_name}_null_count"] = null_count

    # 2. Check Coordinates
    results["coordinates"] = {}
    for coord_name in dataset.coords:
        if dataset.coords[coord_name].dtype == 'datetime64[ns]':
            null_count = np.isnat(dataset.coords[coord_name]).sum()
        else:
            null_count = dataset.coords[coord_name].isnull().sum()
        
        # Check if the result is a Dask array before calling .compute()
        if hasattr(null_count, 'compute'):
            null_count = null_count.compute()
        # null_count = null_count.item() # No longer needed.
        results["coordinates"][coord_name] = null_count > 0
        if null_count > 0:
            results["coordinates"][f"{coord_name}_null_count"] = null_count

    # 3. Check Attributes
    results["attributes"] = {}
    for attr_name, attr_value in dataset.attrs.items():
        results["attributes"][attr_name] = pd.isnull(attr_value)

    return results
check_for_nulls(curated_train)

{'data_variables': {'constants': <xarray.DataArray 'constants' ()> Size: 1B
  array(True),
  'constants_null_count': <xarray.DataArray 'constants' ()> Size: 8B
  array(10382400),
  'forcings': <xarray.DataArray 'forcings' ()> Size: 1B
  array(True),
  'forcings_null_count': <xarray.DataArray 'forcings' ()> Size: 8B
  array(25956000),
  'inputs': <xarray.DataArray 'inputs' ()> Size: 1B
  array(True),
  'inputs_null_count': <xarray.DataArray 'inputs' ()> Size: 8B
  array(742341600),
  'node_features': <xarray.DataArray 'node_features' ()> Size: 1B
  array(True),
  'node_features_null_count': <xarray.DataArray 'node_features' ()> Size: 8B
  array(5191200)},
 'coordinates': {'constants_channel': <xarray.DataArray 'constants_channel' ()> Size: 1B
  array(False),
  'forcings_channel': <xarray.DataArray 'forcings_channel' ()> Size: 1B
  array(False),
  'inputs_channel': <xarray.DataArray 'inputs_channel' ()> Size: 1B
  array(False),
  'latitude': <xarray.DataArray 'latitude' ()> Size: 1B
  ar

In [15]:
curated_train['constants'].sel(constants_channel='land_sea_mask').values

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]],

    